i havent added k fold validation or the scikit learn train test split, but how can i use docker so that the model can be used within an application, and how can i upload the things req for the env to be recreated to github if that is better than using docker

In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import warnings
warnings.filterwarnings("ignore", message="expandable_segments not supported on this platform")

import os
os.environ["MLFLOW_LOCK_MODEL_DEPENDENCIES"] = "true"

import torch
torch.cuda.empty_cache()

import torch
print(torch.__version__)

print(os.cpu_count())

2.5.1+cu118
16


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
from spikingjelly.clock_driven import functional
import json
import random
# from torch.amp import autocast, GradScaler
from contextlib import nullcontext
from typing import Optional
from functools import partial
import traceback


import importlib
import snn_model
import ravdess_dataset
importlib.reload(ravdess_dataset)
importlib.reload(snn_model)
from snn_model import EmotionSNN
from ravdess_dataset import RAVDESSDataset
import mlflow

In [3]:
mlflow.set_tracking_uri("sqlite:///snn.db")
mlflow.set_experiment("SNN_audio-experiment")

2025/09/24 20:28:51 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/24 20:28:51 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='file:///c:/Users/Marc/Desktop/Programming/Main Project/snn/audio-snn/mlruns/1', creation_time=1758724325393, experiment_id='1', last_update_time=1758724325393, lifecycle_stage='active', name='SNN_audio-experiment', tags={}>

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == "cuda":
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

Using device: cuda


In [5]:
# Add this cell early in the notebook
import platform
import sys
import subprocess

def log_environment():
    """Log complete environment for reproducibility"""
    env_info = {
        "python_version": sys.version,
        "platform": platform.platform(),
        "pytorch_version": torch.__version__,
        "cuda_version": torch.version.cuda,
        "cuda_available": torch.cuda.is_available(),
        "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None",
        "spikingjelly_version": getattr(functional, '__version__', 'unknown'),
        "mlflow_version": mlflow.__version__,
        "optuna_version": optuna.__version__,
    }
    
    # Log complete environment as JSON artifact
    mlflow.log_text(json.dumps(env_info, indent=2), "environment.json")
    
    # Log key info as MLflow tags (for filtering/searching)
    mlflow.set_tag("env_pytorch_version", env_info["pytorch_version"])
    mlflow.set_tag("env_cuda_version", str(env_info["cuda_version"]))
    mlflow.set_tag("env_cuda_available", str(env_info["cuda_available"]))
    mlflow.set_tag("env_gpu_name", env_info["gpu_name"])
    mlflow.set_tag("env_platform", env_info["platform"])

    return env_info



In [18]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# PyTorch deterministic flags for reproducibility (may slow training)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)


In [ ]:
dataset_root = "Audio_Speech_Actors_01-24"

# Load dataset
full_dataset = RAVDESSDataset(root_dir=dataset_root, T=50)

train_size = int(0.6 * len(full_dataset))
val_size = int(0.2 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

generator = torch.Generator().manual_seed(SEED)

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, 
    [train_size, val_size, test_size],
    generator=generator
)

# Log split indices for exact reproduction
train_indices = train_dataset.indices
val_indices = val_dataset.indices  
test_indices = test_dataset.indices

print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}, Test size: {len(test_dataset)}")
mlflow.log_text(json.dumps({
    "train_indices": train_indices,
    "val_indices": val_indices,
    "test_indices": test_indices
}, indent=2), "data_splits.json")

Train size: 864, Validation size: 288, Test size: 288


In [8]:
train_dataset[0][0]

tensor([[[[[[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [1., 1., 1.,  ..., 0., 0., 0.],
            [1., 1., 1.,  ..., 0., 0., 0.],
            [1., 1., 1.,  ..., 0., 0., 0.]]],


          [[[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [1., 1., 1.,  ..., 0., 0., 0.],
            [1., 1., 1.,  ..., 0., 0., 0.],
            [1., 1., 1.,  ..., 0., 0., 0.]]],


          [[[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [1., 1., 1.,  ..., 0., 0., 0.],
            [1., 1., 1.,  ..., 0., 0., 0.],
            [1., 1., 1.,  ..., 0., 0., 0.]]],


          ...,


          [[[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0

In [9]:
# quick checks for what full_dataset returns
x, y = full_dataset[0]
print("x.shape:", x.shape, "dtype:", x.dtype, "min/max/mean:", x.min().item(), x.max().item(), x.mean().item())
print("y:", y)

# If full_dataset returns spikes (binary), inspect raw spectrogram:
ds_raw = RAVDESSDataset(root_dir=dataset_root, T=50)
spec, _ = ds_raw[0]
print("raw spec shape, min/max/mean:", spec.shape, spec.min().item(), spec.max().item(), spec.mean().item())

x.shape: torch.Size([50, 1, 50, 1, 128, 400]) dtype: torch.float32 min/max/mean: 0.0 1.0 0.0691642314195633
y: tensor(0)
raw spec shape, min/max/mean: torch.Size([50, 1, 50, 1, 128, 400]) 0.0 1.0 0.06915844976902008


In [10]:
batch = next(iter(DataLoader(train_dataset, batch_size=4)))
data, labels = batch
print("Batch x shape:", data.shape, "dtype:", data.dtype, "min/max:", data.min().item(), "/", data.max().item(), "mean:", data.mean().item())
print("Batch y shape:", labels.shape, "dtype:", labels.dtype, "min/max:", labels.min().item(), "/", labels.max().item(), "mean:", labels.float().mean().item())
print("Sample labels:", labels.tolist())

Batch x shape: torch.Size([4, 50, 1, 50, 1, 128, 400]) dtype: torch.float32 min/max: 0.0 / 1.0 mean: 0.08735780417919159
Batch y shape: torch.Size([4]) dtype: torch.int64 min/max: 2 / 5 mean: 4.0
Sample labels: [5, 5, 2, 4]


In [11]:
print("Input 1:", data[0])
print("Input 2:", data[1])
print("Input 3:", data[2])

Input 1: tensor([[[[[[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [1., 1., 1.,  ..., 0., 0., 0.],
            [1., 1., 1.,  ..., 0., 0., 0.],
            [1., 1., 1.,  ..., 0., 0., 0.]]],


          [[[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [1., 1., 1.,  ..., 0., 0., 0.],
            [1., 1., 1.,  ..., 0., 0., 0.],
            [1., 1., 1.,  ..., 0., 0., 0.]]],


          [[[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [1., 1., 1.,  ..., 0., 0., 0.],
            [1., 1., 1.,  ..., 0., 0., 0.],
            [1., 1., 1.,  ..., 0., 0., 0.]]],


          ...,


          [[[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 

In [12]:
def log_model_with_signature(model, model_name, dataloader, device):
    """Log model with proper signature and input example"""
    model.eval()
    
    # Get a sample batch
    sample_batch = next(iter(dataloader))
    sample_input, sample_target = sample_batch
    
    # Take just one sample and move to device
    sample_input = sample_input[:1].to(device)
    
    with torch.no_grad():
        sample_output = model(sample_input)
    
    # Create signature from numpy arrays
    signature = mlflow.models.signature.infer_signature(
        sample_input.cpu().numpy(), 
        sample_output.cpu().numpy()
    )
    
    # Log model with signature and input example
    mlflow.pytorch.log_model(
        model, 
        model_name,
        signature=signature,
        input_example=sample_input.cpu().numpy()
    )


In [ ]:
@torch.inference_mode()
def validate(model, val_loader, criterion, device):
    model.eval()

    total_loss = 0.0
    total_samples = 0
    total_correct = 0


    for data, target in val_loader:
        data = data.to(device, non_blocking=True)
        target = target.to(device, non_blocking=True)

        # Forward
        outputs = model(data)
        loss = criterion(outputs, target)

        # Update aggregates
        batch_size = target.size(0)
        total_loss += loss.item()
        total_samples += batch_size

        preds = outputs.argmax(dim=1)
        total_correct += (preds == target).sum().item()

        # Reset SNN states to avoid carryover between batches
        functional.reset_net(model)

    # Safeguards
    num_batches = max(1, len(val_loader))
    avg_loss = total_loss / num_batches
    accuracy = 100.0 * (total_correct / max(1, total_samples))
    return avg_loss, accuracy


In [ ]:
def train_model(
    model,
    train_loader,
    val_loader,
    num_epochs,
    learning_rate,
    device,
    patience=10,
    min_delta=0.001,
    ckpt_dir="checkpoints",
    grad_clip_max_norm: Optional[float] = None,
    report_intermediate = True,
    trial = None
):
    
    os.makedirs(ckpt_dir, exist_ok=True)
    
    # advanced optimizer setup
    optimizer = torch.optim.Adam(model.parameters(), 
                                 lr=learning_rate,
                                 weight_decay=0.01,
                                 betas=(0.9, 0.999)
                    )


    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min',
        factor=0.5,
        patience=5,
        verbose=True,
        min_lr=1e-7,
        threshold=min_delta
    )


    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    best_val_loss = float('inf')
    best_val_acc = 0.0
    best_epoch = 0
    train_losses, val_losses, train_accs, val_accs = [], [], [], []
    best_model_state = None
    no_improve_epochs = 0

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss, train_acc = 0.0, 0.0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            
            # Combined loss with L2 regularization
            ce_loss = criterion(output, target)
            l2_loss = model.get_l2_loss()
            loss = ce_loss + l2_loss
            
            loss.backward()
            
            # Gradient clipping to prevent exploding gradients
            if grad_clip_max_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=grad_clip_max_norm)
            else:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            functional.reset_net(model)
            
            train_loss += loss.item()
            pred = output.argmax(dim=1)
            train_acc += pred.eq(target).sum().item()
            
        # ------- Validate -------
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        # ------- Scheduler step -------
        scheduler.step(val_loss)
        
        # ------- Calculate metrics -------
        epoch_train_loss = train_loss / len(train_loader)
        epoch_train_acc = 100.0 * train_acc / len(train_loader.dataset)
        
        train_accs.append(epoch_train_acc)
        val_accs.append(val_acc)
        train_losses.append(epoch_train_loss)
        val_losses.append(val_loss)

        # ------- Track best -------
        val_improved = val_acc > best_val_acc + min_delta
        loss_improved = val_loss < best_val_loss - min_delta
        
        # Update best metrics
        if val_improved:
            best_val_acc = val_acc
            best_epoch = epoch
            no_improvement_epochs = 0
        else:
            no_improvement_epochs += 1
            
        if loss_improved:
            best_val_loss = val_loss
            best_model_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            if trial is None:  # Only save during final training
                best_path = os.path.join(ckpt_dir, "best.pth")
                torch.save(best_model_state, best_path)
                if 'mlflow' in globals():
                    mlflow.log_artifact(best_path, artifact_path="checkpoints")

        # ------- MLFlow Logging -------
        mlflow.log_metric("train_loss", epoch_train_loss, step=epoch)
        mlflow.log_metric("val_loss", val_loss, step=epoch)
        mlflow.log_metric("train_acc", epoch_train_acc, step=epoch)
        mlflow.log_metric("val_acc", val_acc, step=epoch)
        mlflow.log_metric("best_val_acc", best_val_acc, step=epoch)
        mlflow.log_metric("learning_rate", optimizer.param_groups[0]['lr'], step=epoch)
        
        
        # ------- Early stopping (only for final training) -------
        if no_improvement_epochs >= patience:
            print(f'Early stopping at epoch {epoch+1} (no improvement for {patience} epochs)')
            break
        
        
        # -------- Print epoch summary -------
        print(f'Epoch {epoch+1}/{num_epochs}: '
              f'Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, '
              f'Best Val Acc: {best_val_acc:.2f}%, '
              f'LR: {optimizer.param_groups[0]["lr"]:.2e}')

    # Load the best weights seen during this call
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        
    return best_val_acc, best_val_loss, train_accs, val_accs, train_losses, val_losses

In [17]:
def plot_training_curves(train_losses, val_losses, train_accs, val_accs):
    """Plot training and validation curves"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss curves
    ax1.plot(train_losses, label='Train Loss')
    ax1.plot(val_losses, label='Val Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.legend()
    ax1.grid(True)
    
    # Accuracy curves
    ax2.plot(train_accs, label='Train Acc')
    ax2.plot(val_accs, label='Val Acc')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.set_title('Training and Validation Accuracy')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.savefig('training_curves.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
def train_final_model(best_params, num_epochs=100):
    """Train final model with best parameters"""
    
    print("Training final model with best parameters...")
    print("Best parameters:", best_params)
    
    criterion = nn.CrossEntropyLoss()

    
    # Create data loaders
    train_loader = DataLoader(train_dataset, 
                              batch_size=best_params['batch_size'], 
                              shuffle=True, 
                              drop_last=True, 
                              num_workers=8 ,
                              pin_memory=True, 
                              persistent_workers=True,
                              prefetch_factor=2
                    )
    val_loader = DataLoader(val_dataset, 
                            batch_size=best_params['batch_size'], 
                            shuffle=False,        
                            num_workers=8,
                            pin_memory=True,
                            persistent_workers=True,
                            prefetch_factor=2,
                            drop_last=False
                    )
    
    test_loader = DataLoader(test_dataset, 
                             batch_size=best_params['batch_size'], 
                             shuffle=False,        
                             num_workers=8,
                             pin_memory=True,
                             persistent_workers=True,
                             prefetch_factor=2,
                             drop_last=False
                    )
    

    
    # Create model
    model = EmotionSNN(
        num_classes=8,
        conv1_channels=best_params['conv1_channels'],
        conv2_channels=best_params['conv2_channels'],
        fc1_units=best_params['fc1_units'],
        surrogate_func=best_params['surrogate_func'],
        dropout_rate=best_params['dropout_rate']
    ).to(device)

    with mlflow.start_run(run_name="snn_final_training"):
        
        
        # Seeding
        torch.manual_seed(SEED)
        torch.cuda.manual_seed_all(SEED)
        
        mlflow.set_tag("phase", "final")
        mlflow.set_tag("model","EmotionSNN")
        mlflow.log_params(best_params)  # Log hyperparameters
        
        # Save indices as artifacts
        torch.save(train_indices, "train_indices.pt")
        torch.save(val_indices, "val_indices.pt")  
        torch.save(test_indices, "test_indices.pt")
        mlflow.log_artifact("train_indices.pt", "data_splits")
        mlflow.log_artifact("val_indices.pt", "data_splits")
        mlflow.log_artifact("test_indices.pt", "data_splits")
        mlflow.log_artifact("requirements.txt")
        
        # logging environment
        env_info = log_environment()
        
        # Train
        best_val_acc, best_val_loss, train_accs, val_accs, train_losses, val_losses = train_model(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            num_epochs=num_epochs,
            learning_rate=best_params['learning_rate'],
            device=device,
            patience=15
        )
        # Save the trained model
        log_model_with_signature(model, "final_model", train_loader, device)
        
        # Plot training curves
        plot_training_curves(train_losses, val_losses, train_accs, val_accs)
        
        # Test metrics
        test_loss, test_acc = validate(model, test_loader, criterion, device)
        mlflow.log_metric("test_loss", test_loss)
        mlflow.log_metric("test_acc", test_acc)
        
    return model, test_acc

In [ ]:
# Download artifact to a local path
client = mlflow.tracking.MlflowClient()
run_id = "<run_id_with_best_params>"  # Replace with the actual run_id
local_path = client.download_artifacts(run_id, "optuna/best_params.json")

# Load params from JSON
with open(local_path, "r") as f:
    best_params = json.load(f)

# Now use best_params to train your final model
final_model, test_acc = train_final_model(best_params, num_epochs=100)

In [ ]:
final_model, test_acc = train_final_model(best_params, num_epochs=100)